In [2]:
import pandas as pd
import sys
sys.path.insert(0, r"C:/Users/malaka/Documents/prog/BDD")
import fonctions
import psycopg2
from tqdm import tqdm

In [3]:
######## PARAMETRES :: CONNEXION à LA BASE DE DONNES #########
Nom_base = "bdsociohab"
Nom_utilisateur = "admoddh"
mot_de_passe = "adm99!log"
nom_host = "s934"
port = "5441"

suffix_nom_schema = 'aides_personne'
nom_schema_donnees = str('socio_hab_' + suffix_nom_schema)
source = 'DASL'

In [4]:
con = fonctions.connexion(Nom_base, Nom_utilisateur, mot_de_passe, nom_host, port)
table_names = []
cursor = con.cursor()
cursor.execute("SELECT table_name FROM information_schema.tables WHERE table_schema = "+ "'"+nom_schema_donnees+"' AND table_type = 'BASE TABLE'")
        
for table in cursor.fetchall():
    table_names.append(table[0])
table_names

['fsl', 'pel', 'Corres_com_epci']

In [5]:
con = fonctions.connexion(Nom_base, Nom_utilisateur, mot_de_passe, nom_host, port)

for tab in tqdm(range(len(table_names))):
    #print('Vue de la table :: ',table_names[tab])
    #nomenclature 
    view_name = table_names[tab]
    name_avec_schema = nom_schema_donnees+"."+view_name
    table_name=name_avec_schema+"_Vue"
    
    # les modalités présentes dans la table
    sqlQuery_modalites = "select distinct {modalites}.codemodalext "+\
                            "from {modalites}, {name_avec_schema} "+\
                            "where {modalites}.codemodalid = {name_avec_schema}.codemodalid"
    
    sqlQuery_modalites = sqlQuery_modalites.format(name_avec_schema=name_avec_schema, table_name=table_name,
                          modalites= 'modalites_'+suffix_nom_schema.lower(),
                          variables= 'variables_'+suffix_nom_schema.lower(),
                          posseder=  'posseder_'+suffix_nom_schema.lower()   
                                                   
                                                  )
    #print(sqlQuery_modalites)
    try:
        tab_modalites_avec_type = []
        tab_modalites_sans_type = []
        cursor = con.cursor()
        cursor.execute(sqlQuery_modalites)
        result = cursor.fetchall()
        for i in range(len(result)):
                tab_modalites_avec_type.append(str('"'+result[i][0]+'"' + " double precision"))
                tab_modalites_sans_type.append(str(result[i][0]))
        #print(tab_modalites_avec_type)
        #print(tab_modalites_sans_type)
    except (Exception, psycopg2.Error) as error :
        print(sqlQuery_modalites)
    
    con = fonctions.connexion(Nom_base, Nom_utilisateur, mot_de_passe, nom_host, port)
    sqlQuery =  """
             CREATE VIEW {table_name} AS SELECT * FROM crosstab(
                 ' SELECT concat({name_avec_schema}.codeentite, {name_avec_schema}.annee) as row_id, {name_avec_schema}.codeentite, 
                 {name_avec_schema}.annee, {modalites}.codemodalext, {name_avec_schema}.valeur
                FROM  {name_avec_schema}, {modalites}
                 WHERE {modalites}.codemodalid = {name_avec_schema}.codemodalid',
                 $$ values """
    
    # Ajout des valeurs précises
    for i in range(len(tab_modalites_sans_type)):
        if(i == len(result)-1):
            sqlQuery = sqlQuery+ " ('{}') $$".format(tab_modalites_sans_type[i])
        else:
            sqlQuery = sqlQuery+ " ('{}'),".format(tab_modalites_sans_type[i])
    sqlQuery = sqlQuery + ') AS table_fsl ("row_id" text,"codeentite" text, "annee" text,'
#     sqlQuery = sqlQuery.format(name_avec_schema=name_avec_schema, table_name=table_name,
#                           modalites= 'modalites_'+suffix_nom_schema.lower(),
#                           variables= 'variables_'+suffix_nom_schema.lower(),
#                           posseder=  'posseder_'+suffix_nom_schema.lower() )

 
    #print(sqlQuery)
    for i in range(len(tab_modalites_avec_type)):
        if(i == len(result)-1):
            sqlQuery = sqlQuery+ " {}".format(tab_modalites_avec_type[i])
        else:
            sqlQuery = sqlQuery+ " {},".format(tab_modalites_avec_type[i])
    sqlQuery = sqlQuery +");"
    sqlQuery = sqlQuery.format(name_avec_schema=name_avec_schema, table_name=table_name,
                      modalites= 'modalites_'+suffix_nom_schema.lower(),
                      variables= 'variables_'+suffix_nom_schema.lower(),
                      posseder=  'posseder_'+suffix_nom_schema.lower()) 

    try:
        cur = con.cursor()
        dropingLine = 'DROP VIEW  IF EXISTS ' + table_name+ ';'
        cur.execute(dropingLine)
        cur.execute(sqlQuery)
        con.commit()
        print('______________________________________________________________________________')
        print("La Vue "+name_avec_schema+ " à été créée avec succès dans PostgreSQL \n")
        #print(sqlQuery,'\n')
    except (Exception) as error :
        print ("Erreur lors de la création de la Vue"+name_avec_schema+" dans PostgreSQL: ", error)
        print(sqlQuery)
#print(sqlQuery)

100%|██████████| 3/3 [00:00<00:00,  8.77it/s]

______________________________________________________________________________
La Vue socio_hab_aides_personne.fsl à été créée avec succès dans PostgreSQL 

______________________________________________________________________________
La Vue socio_hab_aides_personne.pel à été créée avec succès dans PostgreSQL 

select distinct modalites_aides_personne.codemodalext from modalites_aides_personne, socio_hab_aides_personne.Corres_com_epci where modalites_aides_personne.codemodalid = socio_hab_aides_personne.Corres_com_epci.codemodalid
Erreur lors de la création de la Vuesocio_hab_aides_personne.Corres_com_epci dans PostgreSQL:  ERREUR:  chaîne entre guillemets dollars non terminée sur ou près de « $$ values ) AS table_fsl ("row_id" text,"codeentite" text, "annee" text,); »
LINE 7:                  $$ values ) AS table_fsl ("row_id" text,"co...
                         ^


             CREATE VIEW socio_hab_aides_personne.Corres_com_epci_Vue AS SELECT * FROM crosstab(
                 ' SE